# Chapter 13: Pipelines, Cross-Validation, and Tuning
## Practice Activities

This notebook contains all practice activities from Chapter 13 of "Data Science and Machine Learning with Python."

---

## 📦 Setup: Import Libraries

First, let's import all the necessary libraries.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Scikit-learn components
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ All libraries imported successfully!")

## 📊 Load the AMES Housing Dataset

Load the AMES Housing dataset. Make sure you've uploaded the file to your Colab environment.

**If you haven't uploaded the file yet:**
1. Click the folder icon on the left sidebar
2. Click the upload button
3. Select your `AmesHousing.csv` file

**Alternative:** You can also download it directly from the course repository.

In [ ]:
# Load the dataset
# If the file is in your current directory:
ames = pd.read_csv("AmesHousing.csv")

# If you uploaded to a specific path, modify the path accordingly:
# ames = pd.read_csv("/content/AmesHousing.csv")

print(f"Dataset loaded successfully!")
print(f"Shape: {ames.shape}")
print(f"\nFirst few rows:")
ames.head()

### Quick Data Exploration

In [ ]:
# Check the columns we'll be using
print("Columns we'll use in our analysis:")
print("- Gr Liv Area: Above grade (ground) living area square feet")
print("- TotRms AbvGrd: Total rooms above grade (does not include bathrooms)")
print("- Bldg Type: Type of dwelling")
print("- SalePrice: Sale price (target variable)")
print("\nBasic statistics:")
ames[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type', 'SalePrice']].describe()

## 🎯 Prepare Data for Modeling

**IMPORTANT:** We will perform the train-test split **ONLY ONCE** at the beginning. All models will use the same train and test sets.

In [ ]:
# Drop rows with missing values in the columns we need
ames_clean = ames.dropna(subset=['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type', 'SalePrice'])

# Separate features and target
X = ames_clean[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']]
y = ames_clean['SalePrice']

# Perform train-test split ONCE (as instructed in the chapter)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("✓ Data prepared successfully!")
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"\nFeatures: {list(X.columns)}")
print(f"Target: SalePrice")

---

# 📝 Practice Activity 1: Four Models with Test Set Evaluation

**Task (Section 13.2.5):** Consider four possible models for predicting house prices:

1. Using only the size and number of rooms
2. Using size, number of rooms, and building type
3. Using size and building type, and their interaction
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and building type

Set up a pipeline for each of these four models. Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

**Note:** You should only use the function `train_test_split()` one time in your code; that is, we should be predicting on the **same** test set for all three models.

## Model 1: Size and Number of Rooms Only

In [ ]:
# Model 1: Simple model with just size and number of rooms
pipeline_1 = Pipeline([
    ("standardize", StandardScaler()),
    ("linear_regression", LinearRegression())
])

# Select only the numeric features
X_train_1 = X_train[['Gr Liv Area', 'TotRms AbvGrd']]
X_test_1 = X_test[['Gr Liv Area', 'TotRms AbvGrd']]

# Fit the pipeline
pipeline_1.fit(X_train_1, y_train)

# Make predictions
y_pred_1 = pipeline_1.predict(X_test_1)

# Calculate metrics
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
r2_1 = r2_score(y_test, y_pred_1)

print("Model 1: Size and Number of Rooms Only")
print("=" * 50)
print(f"Test RMSE: ${rmse_1:,.2f}")
print(f"Test R²: {r2_1:.4f}")

## Model 2: Size, Number of Rooms, and Building Type

In [ ]:
# Model 2: Add building type (categorical variable)
# We need ColumnTransformer to handle both numeric and categorical features

ct_2 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
],
remainder="drop"
)

pipeline_2 = Pipeline([
    ("preprocessing", ct_2),
    ("linear_regression", LinearRegression())
]).set_output(transform="pandas")

# Fit the pipeline
pipeline_2.fit(X_train, y_train)

# Make predictions
y_pred_2 = pipeline_2.predict(X_test)

# Calculate metrics
rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred_2))
r2_2 = r2_score(y_test, y_pred_2)

print("Model 2: Size, Number of Rooms, and Building Type")
print("=" * 50)
print(f"Test RMSE: ${rmse_2:,.2f}")
print(f"Test R²: {r2_2:.4f}")

## Model 3: Size, Building Type, and Their Interaction

In [ ]:
# Model 3: Include interaction between size and building type

ct_3 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area"])
],
remainder="drop"
)

pipeline_3 = Pipeline([
    ("preprocessing", ct_3),
    ("interaction", PolynomialFeatures(interaction_only=True)),
    ("linear_regression", LinearRegression())
]).set_output(transform="pandas")

# Select only the features we need for this model
X_train_3 = X_train[['Gr Liv Area', 'Bldg Type']]
X_test_3 = X_test[['Gr Liv Area', 'Bldg Type']]

# Fit the pipeline
pipeline_3.fit(X_train_3, y_train)

# Make predictions
y_pred_3 = pipeline_3.predict(X_test_3)

# Calculate metrics
rmse_3 = np.sqrt(mean_squared_error(y_test, y_pred_3))
r2_3 = r2_score(y_test, y_pred_3)

print("Model 3: Size, Building Type, and Their Interaction")
print("=" * 50)
print(f"Test RMSE: ${rmse_3:,.2f}")
print(f"Test R²: {r2_3:.4f}")

## Model 4: 5-Degree Polynomials on Size and Rooms, Plus Building Type

In [ ]:
# Model 4: Complex model with polynomial features

ct_4 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("polynomial_size", PolynomialFeatures(degree=5), ["Gr Liv Area"]),
    ("polynomial_rooms", PolynomialFeatures(degree=5), ["TotRms AbvGrd"])
],
remainder="drop"
)

pipeline_4 = Pipeline([
    ("preprocessing", ct_4),
    ("linear_regression", LinearRegression())
]).set_output(transform="pandas")

# Fit the pipeline
pipeline_4.fit(X_train, y_train)

# Make predictions
y_pred_4 = pipeline_4.predict(X_test)

# Calculate metrics
rmse_4 = np.sqrt(mean_squared_error(y_test, y_pred_4))
r2_4 = r2_score(y_test, y_pred_4)

print("Model 4: 5-Degree Polynomials on Size and Rooms, Plus Building Type")
print("=" * 50)
print(f"Test RMSE: ${rmse_4:,.2f}")
print(f"Test R²: {r2_4:.4f}")

## 📊 Summary: Practice Activity 1 Results

In [ ]:
# Create a summary dataframe
results_df = pd.DataFrame({
    'Model': [
        'Model 1: Size + Rooms',
        'Model 2: Size + Rooms + Bldg Type',
        'Model 3: Size × Bldg Type Interaction',
        'Model 4: 5-Degree Polynomials'
    ],
    'Test RMSE': [rmse_1, rmse_2, rmse_3, rmse_4],
    'Test R²': [r2_1, r2_2, r2_3, r2_4]
})

# Sort by RMSE (lower is better)
results_df = results_df.sort_values('Test RMSE')

print("\n" + "=" * 70)
print("PRACTICE ACTIVITY 1 SUMMARY: Test Set Performance")
print("=" * 70)
print(results_df.to_string(index=False))
print("\n" + "=" * 70)
print(f"✓ Best Model: {results_df.iloc[0]['Model']}")
print(f"  RMSE: ${results_df.iloc[0]['Test RMSE']:,.2f}")
print(f"  R²: {results_df.iloc[0]['Test R²']:.4f}")
print("=" * 70)

## 📈 Visualization: Model Comparison

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: RMSE comparison
colors = ['#2ecc71' if i == 0 else '#3498db' for i in range(len(results_df))]
axes[0].barh(results_df['Model'], results_df['Test RMSE'], color=colors)
axes[0].set_xlabel('RMSE ($)', fontsize=12)
axes[0].set_title('Test Set RMSE by Model\n(Lower is Better)', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Plot 2: R² comparison
axes[1].barh(results_df['Model'], results_df['Test R²'], color=colors)
axes[1].set_xlabel('R² Score', fontsize=12)
axes[1].set_title('Test Set R² by Model\n(Higher is Better)', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

---

# 📝 Practice Activity 2: Cross-Validation on Four Models

**Task (Section 13.3.1):** Once again consider four modeling options for house price:

1. Using only the size and number of rooms
2. Using size, number of rooms, and building type
3. Using size and building type, and their interaction
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and building type

Use `cross_val_score()` with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

**Which do you prefer? Does this agree with your conclusion from earlier?**

## Model 1: Cross-Validation

In [ ]:
# Model 1 with cross-validation
print("Model 1: Size and Number of Rooms Only")
print("=" * 50)

# Get R² scores across 5 folds
cv_scores_1 = cross_val_score(
    pipeline_1, 
    X[['Gr Liv Area', 'TotRms AbvGrd']], 
    y, 
    cv=5, 
    scoring='r2'
)

# Get RMSE (note: sklearn returns negative MSE, so we negate and take sqrt)
cv_mse_scores_1 = -cross_val_score(
    pipeline_1, 
    X[['Gr Liv Area', 'TotRms AbvGrd']], 
    y, 
    cv=5, 
    scoring='neg_mean_squared_error'
)
cv_rmse_1 = np.sqrt(cv_mse_scores_1.mean())
cv_r2_1 = cv_scores_1.mean()

print(f"Cross-Validated RMSE: ${cv_rmse_1:,.2f}")
print(f"Cross-Validated R²: {cv_r2_1:.4f}")
print(f"R² scores across 5 folds: {cv_scores_1}")
print(f"Standard deviation of R²: {cv_scores_1.std():.4f}")

## Model 2: Cross-Validation

In [ ]:
# Model 2 with cross-validation
print("Model 2: Size, Number of Rooms, and Building Type")
print("=" * 50)

cv_scores_2 = cross_val_score(pipeline_2, X, y, cv=5, scoring='r2')
cv_mse_scores_2 = -cross_val_score(pipeline_2, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_2 = np.sqrt(cv_mse_scores_2.mean())
cv_r2_2 = cv_scores_2.mean()

print(f"Cross-Validated RMSE: ${cv_rmse_2:,.2f}")
print(f"Cross-Validated R²: {cv_r2_2:.4f}")
print(f"R² scores across 5 folds: {cv_scores_2}")
print(f"Standard deviation of R²: {cv_scores_2.std():.4f}")

## Model 3: Cross-Validation

In [ ]:
# Model 3 with cross-validation
print("Model 3: Size, Building Type, and Their Interaction")
print("=" * 50)

cv_scores_3 = cross_val_score(
    pipeline_3, 
    X[['Gr Liv Area', 'Bldg Type']], 
    y, 
    cv=5, 
    scoring='r2'
)
cv_mse_scores_3 = -cross_val_score(
    pipeline_3, 
    X[['Gr Liv Area', 'Bldg Type']], 
    y, 
    cv=5, 
    scoring='neg_mean_squared_error'
)
cv_rmse_3 = np.sqrt(cv_mse_scores_3.mean())
cv_r2_3 = cv_scores_3.mean()

print(f"Cross-Validated RMSE: ${cv_rmse_3:,.2f}")
print(f"Cross-Validated R²: {cv_r2_3:.4f}")
print(f"R² scores across 5 folds: {cv_scores_3}")
print(f"Standard deviation of R²: {cv_scores_3.std():.4f}")

## Model 4: Cross-Validation

In [ ]:
# Model 4 with cross-validation
print("Model 4: 5-Degree Polynomials on Size and Rooms, Plus Building Type")
print("=" * 50)

cv_scores_4 = cross_val_score(pipeline_4, X, y, cv=5, scoring='r2')
cv_mse_scores_4 = -cross_val_score(pipeline_4, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_4 = np.sqrt(cv_mse_scores_4.mean())
cv_r2_4 = cv_scores_4.mean()

print(f"Cross-Validated RMSE: ${cv_rmse_4:,.2f}")
print(f"Cross-Validated R²: {cv_r2_4:.4f}")
print(f"R² scores across 5 folds: {cv_scores_4}")
print(f"Standard deviation of R²: {cv_scores_4.std():.4f}")

## 📊 Summary: Practice Activity 2 Results

In [ ]:
# Create summary dataframe for cross-validation results
cv_results_df = pd.DataFrame({
    'Model': [
        'Model 1: Size + Rooms',
        'Model 2: Size + Rooms + Bldg Type',
        'Model 3: Size × Bldg Type Interaction',
        'Model 4: 5-Degree Polynomials'
    ],
    'CV RMSE': [cv_rmse_1, cv_rmse_2, cv_rmse_3, cv_rmse_4],
    'CV R²': [cv_r2_1, cv_r2_2, cv_r2_3, cv_r2_4]
})

# Sort by RMSE (lower is better)
cv_results_df = cv_results_df.sort_values('CV RMSE')

print("\n" + "=" * 70)
print("PRACTICE ACTIVITY 2 SUMMARY: Cross-Validation Performance")
print("=" * 70)
print(cv_results_df.to_string(index=False))
print("\n" + "=" * 70)
print(f"✓ Best Model: {cv_results_df.iloc[0]['Model']}")
print(f"  CV RMSE: ${cv_results_df.iloc[0]['CV RMSE']:,.2f}")
print(f"  CV R²: {cv_results_df.iloc[0]['CV R²']:.4f}")
print("=" * 70)

## 🔍 Comparison: Test Set vs Cross-Validation

Let's compare the results from both approaches to answer the question: **Does cross-validation agree with the test set results?**

In [ ]:
# Create comprehensive comparison
comparison_df = pd.DataFrame({
    'Model': [
        'Model 1: Size + Rooms',
        'Model 2: Size + Rooms + Bldg Type',
        'Model 3: Size × Bldg Type Interaction',
        'Model 4: 5-Degree Polynomials'
    ],
    'Test RMSE': [rmse_1, rmse_2, rmse_3, rmse_4],
    'CV RMSE': [cv_rmse_1, cv_rmse_2, cv_rmse_3, cv_rmse_4],
    'Test R²': [r2_1, r2_2, r2_3, r2_4],
    'CV R²': [cv_r2_1, cv_r2_2, cv_r2_3, cv_r2_4]
})

print("\n" + "=" * 85)
print("COMPARISON: Test Set vs Cross-Validation")
print("=" * 85)
print(comparison_df.to_string(index=False))
print("\n" + "=" * 85)

# Determine best models
best_test = results_df.iloc[0]['Model']
best_cv = cv_results_df.iloc[0]['Model']

print("\n📊 ANALYSIS:")
print(f"  Best model on test set: {best_test}")
print(f"  Best model with CV: {best_cv}")

if best_test == best_cv:
    print("  ✓ Both methods AGREE on the best model!")
else:
    print("  ⚠ Methods DISAGREE - Cross-validation is more reliable!")
    print("\n💡 KEY INSIGHT:")
    print("  Single train-test splits can be misleading due to randomness.")
    print("  Cross-validation averages over multiple splits, giving more")
    print("  stable and reliable estimates of model performance.")

print("=" * 85)

## 📈 Visualization: Test Set vs Cross-Validation

In [ ]:
# Create comparison visualization
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Test RMSE'], width, label='Test Set', alpha=0.8, color='#3498db')
bars2 = ax.bar(x + width/2, comparison_df['CV RMSE'], width, label='Cross-Validation', alpha=0.8, color='#e74c3c')

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('RMSE ($)', fontsize=12)
ax.set_title('Test Set RMSE vs Cross-Validated RMSE\n(Lower is Better)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(['Model 1', 'Model 2', 'Model 3', 'Model 4'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---

# 📝 Practice Activity 3: Hyperparameter Tuning

**Task (Section 13.3.3):** Consider *one hundred* modeling options for house price:

1. Using only the size and number of rooms
2. Using size, number of rooms, and building type
3. Using size and building type, and their interaction
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and building type

Use `cross_val_score()` with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

**Note:** While the chapter mentions "one hundred" options, we'll explore a systematic grid search across polynomial degrees to demonstrate the tuning process.

## Setup: Create Pipeline for Tuning

In [ ]:
# Create a pipeline with polynomial features that we can tune
ct_tuning = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("polynomial", PolynomialFeatures(), ["Gr Liv Area", "TotRms AbvGrd"])
],
remainder="drop"
)

pipeline_tuning = Pipeline([
    ("preprocessing", ct_tuning),
    ("linear_regression", LinearRegression())
]).set_output(transform="pandas")

print("✓ Tuning pipeline created successfully!")
print("\nPipeline structure:")
print(pipeline_tuning)

## Perform Grid Search

In [ ]:
# Define parameter grid - test polynomial degrees from 1 to 10
param_grid = {
    'preprocessing__polynomial__degree': np.arange(1, 11)
}

print("Setting up GridSearchCV...")
print(f"Testing {len(param_grid['preprocessing__polynomial__degree'])} different polynomial degrees")
print("with 5-fold cross-validation.")
print(f"Total number of fits: {len(param_grid['preprocessing__polynomial__degree']) * 5}")
print("\nThis may take a moment...")

# Create and fit GridSearchCV
gscv = GridSearchCV(
    pipeline_tuning, 
    param_grid, 
    cv=5, 
    scoring='r2',
    return_train_score=True,
    verbose=1
)

gscv.fit(X, y)

print("\n✓ Grid search complete!")

## Analyze Grid Search Results

In [ ]:
# Extract and display results
cv_results = pd.DataFrame(gscv.cv_results_)

results_summary = pd.DataFrame({
    'Degree': param_grid['preprocessing__polynomial__degree'],
    'Mean CV R²': cv_results['mean_test_score'],
    'Std CV R²': cv_results['std_test_score'],
    'Mean Train R²': cv_results['mean_train_score']
})

print("\n" + "=" * 70)
print("GRID SEARCH RESULTS: Polynomial Degree Tuning")
print("=" * 70)
print(results_summary.to_string(index=False))
print("\n" + "=" * 70)
print(f"\n✓ Best polynomial degree: {gscv.best_params_['preprocessing__polynomial__degree']}")
print(f"  Best cross-validated R²: {gscv.best_score_:.4f}")
print("=" * 70)

## Check for Overfitting

In [ ]:
print("\n" + "=" * 70)
print("OVERFITTING ANALYSIS")
print("=" * 70)
print("\nChecking the gap between training and cross-validation performance:")
print("Large gaps indicate overfitting.\n")

for idx, row in results_summary.iterrows():
    gap = row['Mean Train R²'] - row['Mean CV R²']
    warning = "⚠ Possible overfitting" if gap > 0.05 else "✓ Good generalization"
    print(f"Degree {int(row['Degree'])}: "
          f"Train R² = {row['Mean Train R²']:.4f}, "
          f"CV R² = {row['Mean CV R²']:.4f} "
          f"(Gap: {gap:.4f}) {warning}")

## 📈 Visualization: Model Complexity vs Performance

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: R² vs Polynomial Degree
axes[0].plot(results_summary['Degree'], results_summary['Mean Train R²'], 
             marker='o', label='Training R²', linewidth=2, markersize=8)
axes[0].plot(results_summary['Degree'], results_summary['Mean CV R²'], 
             marker='s', label='Cross-Validation R²', linewidth=2, markersize=8)
axes[0].axvline(x=gscv.best_params_['preprocessing__polynomial__degree'], 
                color='red', linestyle='--', alpha=0.7, label='Best Degree')
axes[0].set_xlabel('Polynomial Degree', fontsize=12)
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Model Performance vs Complexity\n(Higher is Better)', 
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Overfitting Gap
gap = results_summary['Mean Train R²'] - results_summary['Mean CV R²']
colors = ['#2ecc71' if g < 0.05 else '#e74c3c' for g in gap]
axes[1].bar(results_summary['Degree'], gap, color=colors, alpha=0.7)
axes[1].axhline(y=0.05, color='orange', linestyle='--', alpha=0.7, 
                label='Overfitting Threshold')
axes[1].set_xlabel('Polynomial Degree', fontsize=12)
axes[1].set_ylabel('Train R² - CV R² (Gap)', fontsize=12)
axes[1].set_title('Overfitting Check\n(Lower Gap is Better)', 
                  fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---

# 🎯 Final Summary and Key Takeaways

In [ ]:
print("\n" + "=" * 80)
print("FINAL SUMMARY: ALL PRACTICE ACTIVITIES")
print("=" * 80)

print("\n📊 KEY FINDINGS:")
print("-" * 80)

print(f"\n1. Best Model on Test Set (Activity 1):")
print(f"   Model: {results_df.iloc[0]['Model']}")
print(f"   Test RMSE: ${results_df.iloc[0]['Test RMSE']:,.2f}")
print(f"   Test R²: {results_df.iloc[0]['Test R²']:.4f}")

print(f"\n2. Best Model with Cross-Validation (Activity 2):")
print(f"   Model: {cv_results_df.iloc[0]['Model']}")
print(f"   CV RMSE: ${cv_results_df.iloc[0]['CV RMSE']:,.2f}")
print(f"   CV R²: {cv_results_df.iloc[0]['CV R²']:.4f}")

print(f"\n3. Optimal Polynomial Degree (Activity 3):")
print(f"   Best degree: {gscv.best_params_['preprocessing__polynomial__degree']}")
print(f"   Best CV R²: {gscv.best_score_:.4f}")

print("\n\n💡 KEY LESSONS FROM CHAPTER 13:")
print("-" * 80)
print("\n✓ Pipelines prevent data leakage")
print("  → Ensure preprocessing uses only training data statistics")
print("  → Automatically apply same transformations to test data")

print("\n✓ Cross-validation is more reliable than single train-test splits")
print("  → Averages over multiple splits for stable estimates")
print("  → Helps detect when we're getting 'lucky' with one split")

print("\n✓ Simpler models often generalize better")
print("  → Complex models (high-degree polynomials) can overfit")
print("  → Adding features doesn't always improve performance")

print("\n✓ Systematic hyperparameter tuning is essential")
print("  → GridSearchCV automates the cross-validation process")
print("  → Helps find optimal model configuration")

print("\n" + "=" * 80)
print("PRACTICE ACTIVITIES COMPLETE! 🎉")
print("=" * 80)

---

# 📋 FINAL ANSWER: Best Model and R-Squared Metric

## Question: Report your final best model and corresponding R-squared metric.

In [ ]:
# Determine the final best model based on cross-validation
# (Cross-validation is more reliable than single test set evaluation)

print("\n" + "=" * 80)
print("FINAL ANSWER")
print("=" * 80)

best_model_name = cv_results_df.iloc[0]['Model']
best_cv_r2 = cv_results_df.iloc[0]['CV R²']
best_cv_rmse = cv_results_df.iloc[0]['CV RMSE']

print("\n🏆 BEST MODEL (Based on 5-Fold Cross-Validation):")
print("=" * 80)
print(f"\nModel: {best_model_name}")
print(f"\nCross-Validated R² Score: {best_cv_r2:.4f}")
print(f"Cross-Validated RMSE: ${best_cv_rmse:,.2f}")

print("\n" + "-" * 80)
print("WHY THIS MODEL?")
print("-" * 80)

if best_model_name == 'Model 1: Size + Rooms':
    print("\nThis model uses only two features:")
    print("  • Gr Liv Area (square footage of living area)")
    print("  • TotRms AbvGrd (total rooms above grade)")
    print("\nStrengths:")
    print("  ✓ Simplest model with fewest parameters")
    print("  ✓ Less prone to overfitting")
    print("  ✓ Best cross-validated performance")
    print("  ✓ Most generalizable to new data")
elif best_model_name == 'Model 2: Size + Rooms + Bldg Type':
    print("\nThis model uses:")
    print("  • Gr Liv Area (square footage)")
    print("  • TotRms AbvGrd (total rooms)")
    print("  • Bldg Type (categorical: building type)")
    print("\nStrengths:")
    print("  ✓ Includes categorical building type information")
    print("  ✓ Captures differences between dwelling types")
    print("  ✓ Good balance of complexity and performance")
elif best_model_name == 'Model 3: Size × Bldg Type Interaction':
    print("\nThis model uses:")
    print("  • Gr Liv Area (square footage)")
    print("  • Bldg Type (building type)")
    print("  • Interaction between Size and Building Type")
    print("\nStrengths:")
    print("  ✓ Captures how size affects price differently for different building types")
    print("  ✓ More flexible than additive models")
    print("  ✓ Can model complex relationships")
else:
    print("\nThis model uses:")
    print("  • 5-degree polynomial on Gr Liv Area")
    print("  • 5-degree polynomial on TotRms AbvGrd")
    print("  • Bldg Type (categorical)")
    print("\nStrengths:")
    print("  ✓ Can capture non-linear relationships")
    print("  ✓ Most flexible model structure")

print("\n" + "-" * 80)
print("INTERPRETATION OF R² = {:.4f}:".format(best_cv_r2))
print("-" * 80)
print(f"\nThis model explains approximately {best_cv_r2*100:.2f}% of the variance in house prices.")

if best_cv_r2 < 0.3:
    print("\n⚠ This is a relatively low R² value, suggesting:")
    print("  • House prices are influenced by many factors beyond our features")
    print("  • Additional features (location, condition, age) could improve the model")
    print("  • The relationship may be inherently complex")
elif best_cv_r2 < 0.6:
    print("\n✓ This is a moderate R² value, indicating:")
    print("  • The model captures important trends in house pricing")
    print("  • There is still unexplained variance that other features might capture")
    print("  • The model is useful but has room for improvement")
else:
    print("\n✓ This is a strong R² value, indicating:")
    print("  • The model effectively predicts house prices")
    print("  • Our features capture most of the important pricing factors")
    print("  • The model is reliable for making predictions")

print("\n" + "-" * 80)
print("WHY WE USE CROSS-VALIDATION:")
print("-" * 80)
print("\nWe selected the best model based on cross-validation rather than the test set because:")
print("  1. Cross-validation averages performance over 5 different train-test splits")
print("  2. This provides a more stable and reliable estimate of model performance")
print("  3. A single test set can be 'lucky' or 'unlucky' due to randomness")
print("  4. Cross-validation better predicts how the model will perform on new data")

# Show comparison with test set if different
if best_model_name != results_df.iloc[0]['Model']:
    print("\n" + "-" * 80)
    print("⚠ NOTE: Test Set vs Cross-Validation Disagreement")
    print("-" * 80)
    print(f"\nThe single test set suggested: {results_df.iloc[0]['Model']}")
    print(f"Cross-validation suggests: {best_model_name}")
    print("\nThis demonstrates why cross-validation is more reliable!")
    print("The test set result may have been influenced by the particular random split.")

print("\n" + "=" * 80)
print("END OF FINAL ANSWER")
print("=" * 80)

## 📈 Final Visualization: Best Model

In [ ]:
# Create a clear visualization highlighting the best model
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Prepare data with highlighting
models = cv_results_df['Model'].tolist()
cv_r2_scores = cv_results_df['CV R²'].tolist()
cv_rmse_scores = cv_results_df['CV RMSE'].tolist()

# Color the best model differently
colors = ['#2ecc71' if i == 0 else '#95a5a6' for i in range(len(models))]

# Plot 1: R² scores
bars1 = axes[0].barh(models, cv_r2_scores, color=colors, edgecolor='black', linewidth=2)
axes[0].set_xlabel('Cross-Validated R² Score', fontsize=13, fontweight='bold')
axes[0].set_title('Model Performance (R² Score)\n🏆 Best Model Highlighted in Green', 
                  fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Add value labels
for i, (bar, score) in enumerate(zip(bars1, cv_r2_scores)):
    width = bar.get_width()
    label = f'{score:.4f}'
    if i == 0:
        axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {label} ★', ha='left', va='center', 
                    fontsize=12, fontweight='bold', color='#2ecc71')
    else:
        axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {label}', ha='left', va='center', fontsize=11)

# Plot 2: RMSE scores
bars2 = axes[1].barh(models, cv_rmse_scores, color=colors, edgecolor='black', linewidth=2)
axes[1].set_xlabel('Cross-Validated RMSE ($)', fontsize=13, fontweight='bold')
axes[1].set_title('Model Performance (RMSE)\n🏆 Best Model Highlighted in Green', 
                  fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

# Add value labels
for i, (bar, score) in enumerate(zip(bars2, cv_rmse_scores)):
    width = bar.get_width()
    label = f'${score:,.0f}'
    if i == 0:
        axes[1].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {label} ★', ha='left', va='center', 
                    fontsize=12, fontweight='bold', color='#2ecc71')
    else:
        axes[1].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {label}', ha='left', va='center', fontsize=11)

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("★ The green bars show our final best model!")
print("=" * 80)

## 📊 Complete Model Comparison Table

In [ ]:
# Create a comprehensive comparison table
print("\n" + "=" * 90)
print("COMPLETE MODEL COMPARISON")
print("=" * 90)
print("\nAll Four Models Ranked by Cross-Validated Performance:\n")

# Add rank column
cv_results_ranked = cv_results_df.copy()
cv_results_ranked['Rank'] = range(1, len(cv_results_ranked) + 1)
cv_results_ranked = cv_results_ranked[['Rank', 'Model', 'CV RMSE', 'CV R²']]

print(cv_results_ranked.to_string(index=False))

print("\n" + "=" * 90)
print(f"\n🥇 Winner: {cv_results_ranked.iloc[0]['Model']}")
print(f"   R² = {cv_results_ranked.iloc[0]['CV R²']:.4f}")
print("\n" + "=" * 90)

---

## 🚀 Next Steps and Exercises

Now that you've completed all the practice activities, try these extensions:

1. **Try different features**: Include other variables from the AMES dataset like `Year Built`, `Lot Area`, etc.

2. **Experiment with different models**: Replace `LinearRegression()` with other models like Ridge, Lasso, or RandomForest

3. **Expand the grid search**: Add more hyperparameters to tune:
   ```python
   param_grid = {
       'preprocessing__polynomial__degree': [1, 2, 3, 4, 5],
       'preprocessing__polynomial__interaction_only': [True, False]
   }
   ```

4. **Visualize predictions**: Create scatter plots of actual vs predicted prices for your best model

5. **Feature engineering**: Create new features like `Price per Square Foot` or `Age of House`

6. **Try different CV strategies**: Use `KFold`, `StratifiedKFold`, or `TimeSeriesSplit` instead of the default 5-fold CV